
Control system dataset for causal discovery
===========================================

*K.Rathsman*, *S.W.Mogensen* and *P.Nilsson*

This dataset comprises control system data from the accelerator cryogenics plant (ACCP) at the European Spallation Source (ESS). If you use the data, please cite 

S.W.Mogensen, K.Rathsman, P.Nilsson, Causal discovery in a complex industrial system: A time series benchmark, in Proceedings of the 3rd Conference on Causal Learning and Reasoning (CLeaR), 2024, Available: https://doi.org/10.48550/arXiv.2310.18654
 
The paper also contains a detailed description of the data and of the underlying system. More description, code, and help to get started can be found at https://soerenwengel.github.io/essdata

The following is a brief description of the dataset as well as Python code to get started. The above paper contains a more detailed description. The dataset is intended as a causal discovery benchmark: A causal graph has been constructed based on knowledge of how the ACCP works, and this graph is found in the paper. Causal discovery methods should be able to recover this graph, and different methods can be compared using this benchmark. The exact interpretation of the causal graph is described in more detail in the above paper and its references, and a brief description of the causal graph is given in the Subsystems section below.

The data was recorded during three periods of steady state operation. During each period, the control room operators did not change any of the control settings, however, control settings are different between time periods. A total of 233 different process variables (PVs) are present in the dataset. They are all measurements of physical quantities in the ACCP, e.g., temperatures and pressures. Each observation contains a measured value, a PV index as well as some metadata.

# DataportalClient

In [ ]:
import pandas as pd
from dataportal import DataportalClient;


token = # Enter your generated JWT

client = DataportalClient(token)


# Data files
The dataset consists of a large number of datafiles. To tabulate the files we use the DataPortal.listFiles() method and Pandas:

In [ ]:
%%capture
dataset = 'ControlSystem'
file_iterator = client.listFiles()
file_list = list(file_iterator)
files = pd.DataFrame.from_records(file_list, index = 'FileID')

## Operation periods
The data was recorded during three periods of steady state operation. During each period, the control room operators did not change any of the control settings, however, control settings are different between time periods. Therefore we need to separate the files into three separate lists.

The operation period is indicated by the OriginName in the file list:


In [ ]:
operation_periods = files[['OriginName']].drop_duplicates()
operation_periods

To create separate file lists for the three operation periods, we use the query function:

In [ ]:
files_1 = files.query('OriginName=="operation-period-1"')
files_2 = files.query('OriginName=="operation-period-2"')
files_3 = files.query('OriginName=="operation-period-3"')

files_1

## Initial States
The first data files in each operation period have only a few entries (see MetricEntries) and are not complete. They have been included only to provide initial states to each process variable. To separate out these files we again use the query function:

In [ ]:
files = files_1

files_initial_states = files.query('MetricEntries<=233')
files = files.query('MetricEntries>233')
files_initial_states

# Data

## Initial states
We concatenate the intial states into a dataframe for later use:

In [ ]:
initial_states = pd.concat([client.getData(file_id) for file_id in files_initial_states.index], axis = 0)
for c in initial_states.columns[1:]:
    initial_states[c] = pd.Categorical(initial_states[c], ordered = True)
initial_states.head()
dtypes = initial_states.dtypes.to_dict()

## Metadata 
A total of 233 different process variables (PVs) are present in the dataset. Each process variable has a name, unit, description, subsystem and sensor type. A table of metadata can be created from the intial states as:

In [ ]:
metadata = initial_states.drop('Value',axis='columns').set_index('Name').sort_values(['Subsystem_Index','Name'])
metadata

### Subsystems
The accelerator cryogenics plant system is divided into a number of subsystems. These subsystems and their causal relations are described in the paper mentioned above. To create a table of all subsystems, including the number of process variables they represent we use pandas.value_counts():

In [ ]:
subsystems = metadata[['Subsystem_Index','Subsystem_Description']].value_counts().to_frame(name='Count').reset_index('Subsystem_Description').sort_index()
subsystems

### Sensor types
Process variables represent data from different sensor types, as described in the aforementioned paper. They can be listed as:

In [ ]:
sensor_types = metadata[['SensorType_Index','SensorType_Description']].value_counts().to_frame(name='Count').reset_index(['SensorType_Description' ]).sort_index()
sensor_types

## The actual data
Each row in a datafile represents a state change of exactly one process variable at the time:

In [ ]:
file_id = files.index[0]
data = client.getData(file_id)
data

### Include initial states
This step will insert the initial states into the dataframe and also create new intial states for the next data file.

In [ ]:
start = pd.Index([data.index[0].floor('h')]*len(initial_states), name = data.index.name)
data = pd.concat([initial_states.set_index(start),data.astype(dtypes)])
initial_states = data.loc[data['Name'].drop_duplicates(keep='last').index,:].copy()

data

### Plot
To select and plot the measured values for, e.g., the PV 'TT-34750' from '2022-12-30 17:00' to '2022-12-30 17:10' we use pandas.plot() and populate it with some metadata:

In [ ]:
NAME = 'TT-34750'
FROM = '2022-12-30 17:00'
TO = '2022-12-30 17:10'

query = f"Name == '{NAME}'"
ylabel = f"{metadata.loc[NAME,'Description']} ({metadata.loc[NAME,'Unit']})"

value = data.query(query).loc[FROM:TO,'Value']
ax = value.plot(ylabel=ylabel, title=NAME, marker = '.', linestyle = '', alpha = 0.2)

### Resample
Changes in cryogenics systems are expected to be somewhat slow and to resample the dataset to equidistant timestamps we use Pandas.resample() method. (Note however that resampling and aggregation over time may be problematic in the context of causal discovery.)

In [ ]:
SAMPLING_PERIOD = '10 seconds'

period = pd.Timedelta(SAMPLING_PERIOD)
resampled_data = data.groupby('Name',observed=True)[['Value']].resample(period).mean().reset_index('Name').sort_index()

resampled_value = resampled_data.query(query).loc[FROM:TO,'Value']
legend = f'{SAMPLING_PERIOD=}'

ax = value.plot(ylabel=ylabel, title=NAME, marker = '.', linestyle = '', alpha = 0.2, label = 'raw_data', legend=True)
ax = resampled_value.plot(label = legend, legend=True)

### Pivot
To create a table with a common time index we use Pandas.pivot():

In [ ]:
pivoted_data = resampled_data.pivot(columns = 'Name',values='Value').ffill()

columns = metadata['Subsystem_Index'].sort_values()
pivoted_data  = pivoted_data[list(columns.index)]

pivoted_data.columns = pd.MultiIndex.from_arrays([columns,columns.index])
pivoted_data

### Correlation Matrix
Finally, we can construct the correlation matrix. (Note that this ignores the temporal structure of the data)

In [ ]:

correlation = pivoted_data.corr()
correlation